In [ ]:
!pip install transformers datasets tf-models-official

In [ ]:
from psutil import virtual_memory
from google.colab import files
import pandas as pd
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from transformers import DataCollatorWithPadding
from transformers import TFAutoModel
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from datasets import Dataset
import tensorflow as tf
import tensorflow_models as tfm
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import AdamW, Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
uploaded = files.upload()

Saving train_task2_robertuito.csv to train_task2_robertuito.csv
Saving val_task2_robertuito.csv to val_task2_robertuito.csv


In [ ]:
train_df = pd.read_csv(io.BytesIO(uploaded['train_task2_robertuito.csv']))
val_df = pd.read_csv(io.BytesIO(uploaded['val_task2_robertuito.csv']))

In [ ]:
train_df

,Unnamed: 0,G,L,B,T,O,train_texts
0,18,0,1,0,0,0,"Las famosas ""lesbianas"" no llegan a ser del to..."
1,468,1,0,0,0,0,@usuario tu padre es pendejos tu pendeja marica
2,816,1,0,0,0,0,Puto maricon!!! hashtag rayados no sale campeon
3,402,1,0,0,0,0,Yo le hacia el cambio de sexo gratis a hashtag...
4,640,1,0,0,0,0,"No sean maricas, cuando ustedes redactan spoil..."
...,...,...,...,...,...,...,...
727,534,1,0,0,0,0,@usuario la cita no es ni marica ni de adolori...
728,584,1,0,0,0,0,"Manny Pacquiao: "" hashtag homosexuales son peo..."
729,493,1,0,0,0,0,Respondiendo a la pregunta: solo deja de sacar...
730,527,1,0,0,0,0,emoji bandera españa emoji Soy homosexual\n em...


In [ ]:
label_cols = ['G','L','B','T','O']
train_df['labels'] = list(train_df[label_cols].values)
val_df['labels'] = list(val_df[label_cols].values)

In [ ]:
train_df.head()

,Unnamed: 0,G,L,B,T,O,train_texts,labels
0,18,0,1,0,0,0,"Las famosas ""lesbianas"" no llegan a ser del to...","[0, 1, 0, 0, 0]"
1,468,1,0,0,0,0,@usuario tu padre es pendejos tu pendeja marica,"[1, 0, 0, 0, 0]"
2,816,1,0,0,0,0,Puto maricon!!! hashtag rayados no sale campeon,"[1, 0, 0, 0, 0]"
3,402,1,0,0,0,0,Yo le hacia el cambio de sexo gratis a hashtag...,"[1, 0, 0, 0, 0]"
4,640,1,0,0,0,0,"No sean maricas, cuando ustedes redactan spoil...","[1, 0, 0, 0, 0]"


In [ ]:
train_texts, val_texts, train_labels, val_labels = list(train_df['train_texts']), list(val_df['val_texts']), train_df['labels'], val_df['labels']

In [ ]:
print(len(train_texts))
print(len(val_texts))

732
130


In [ ]:
checkpoint = "pysentimiento/robertuito-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
train_encodings['labels'] = train_labels
val_encodings['labels'] = val_labels

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
batch_size = 4
num_epochs = 7

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = Dataset.from_dict(train_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size
)

tf_validation_dataset = Dataset.from_dict(val_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=2e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = AdamW(learning_rate=lr_scheduler, amsgrad=False)
loss=BinaryCrossentropy(from_logits=True)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5, problem_type="multi_label_classification",
                                                             attention_probs_dropout_prob=0.1,
                                                             hidden_dropout_prob=0.1,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=["accuracy"]
)

In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)

Epoch 1/5
183/183 [==============================] - 80s 187ms/step - loss: 0.0204 - accuracy: 0.9617 - val_loss: 0.1153 - val_accuracy: 0.9462
Epoch 2/5
183/183 [==============================] - 15s 85ms/step - loss: 0.0197 - accuracy: 0.9563 - val_loss: 0.1059 - val_accuracy: 0.9538
Epoch 3/5
183/183 [==============================] - 14s 76ms/step - loss: 0.0136 - accuracy: 0.9631 - val_loss: 0.0992 - val_accuracy: 0.9462
Epoch 4/5
183/183 [==============================] - 13s 73ms/step - loss: 0.0101 - accuracy: 0.9604 - val_loss: 0.1050 - val_accuracy: 0.9538
Epoch 5/5
183/183 [==============================] - 13s 69ms/step - loss: 0.0083 - accuracy: 0.9577 - val_loss: 0.1060 - val_accuracy: 0.9615


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("cybersyn/robertuito-homomex-track2", from_pt=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

tf_model.h5:   0%|          | 0.00/435M [00:00<?, ?B/s]

In [ ]:
tokenizer.push_to_hub("cybersyn/robertuito-homomex-track2")

CommitInfo(commit_url='https://huggingface.co/cybersyn/robertuito-homomex-track2/commit/c0d693016f59b224b96c777ab3bb81c0ae55da85', commit_message='Upload tokenizer', commit_description='', oid='c0d693016f59b224b96c777ab3bb81c0ae55da85', pr_url=None, pr_revision=None, pr_num=None)

###Download and test de model

In [ ]:
del model

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("cybersyn/robertuito-homomex-track2", num_labels=5)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cybersyn/robertuito-homomex-track2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]
class_preds = [(r > 0).astype(np.int) for r in preds]

In [ ]:
print("Robertuito Accuracy Score:", round(accuracy_score(class_preds, list(val_encodings['labels']))*100, 2))

Robertuito Accuracy Score: 87.69


In [ ]:
print(classification_report(class_preds, list(val_encodings['labels'])))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       108
           1       0.67      1.00      0.80         6
           2       0.00      0.00      0.00         0
           3       0.77      0.91      0.83        11
           4       0.00      0.00      0.00         0

   micro avg       0.86      0.98      0.91       125
   macro avg       0.48      0.58      0.52       125
weighted avg       0.93      0.98      0.95       125
 samples avg       0.91      0.94      0.92       125



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
multilabel_confusion_matrix(class_preds, list(val_encodings['labels']))

array([[[ 18,   4],
        [  2, 106]],

       [[121,   3],
        [  0,   6]],

       [[128,   2],
        [  0,   0]],

       [[116,   3],
        [  1,  10]],

       [[122,   8],
        [  0,   0]]])

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("cybersyn/robertuito-homomex-track2", num_labels=5)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cybersyn/robertuito-homomex-track2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
uploaded = files.upload()

Saving pred_task2_robertuito.csv to pred_task2_robertuito.csv


In [ ]:
pred_df = pd.read_csv(io.BytesIO(uploaded['pred_task2_robertuito.csv']))

In [ ]:
pred_df

,Unnamed: 0,0
0,0,"Ash !! La jotita del programa ""hoy"" no deja de..."
1,1,En una boda gay ¿Les avientan arroz con popote?
2,2,@usuario Hazlo mariquita jaja. Hazlo neta! No ...
3,3,"""@usuario: Y el que no habla de la Miss univer..."
4,4,"Vamos Dinamarca, ya empieza la marica de CR7 a..."
...,...,...
472,472,Deje la presidencia a una mujer. Que haya 7 mi...
473,473,@usuario @usuario KHA?\n\nAhora que. No vayas ...
474,474,Lia Thomson (mujer transgénero) logra su prime...
475,475,"""La gente no binarie puede no usar reloj, usar..."


In [ ]:
pred_texts = list(pred_df['0'])

In [ ]:
checkpoint = "pysentimiento/robertuito-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
pred_encodings = tokenizer(pred_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
pred_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_prediction_dataset = Dataset.from_dict(pred_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size
)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
preds = model.predict(tf_prediction_dataset)["logits"]
class_preds = [(r > 0).astype(np.int) for r in preds]

120/120 [==============================] - 6s 23ms/step


<ipython-input-203-313774f735b1>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  class_preds = [(r > 0).astype(np.int) for r in preds]


In [ ]:
l_labels = ['L' if l[1] == 1 else 'X' for l in class_preds]
g_labels = ['G' if l[0] == 1 else 'X' for l in class_preds]
b_labels = ['B' if l[2] == 1 else 'X' for l in class_preds]
t_labels = ['T' if l[3] == 1 else 'X' for l in class_preds]
o_labels = ['O' if l[4] == 1 else 'X' for l in class_preds]

In [ ]:
pred_df['l'] = l_labels
pred_df['g'] = g_labels
pred_df['b'] = b_labels
pred_df['t'] = t_labels
pred_df['o'] = o_labels

In [ ]:
pred_df['TaskName'] = 'LGBTphobiaDetectionMultiLabeled'

In [ ]:
pred_df = pred_df[['TaskName', 'Unnamed: 0', 'l', 'g', 'b', 't', 'o']]

In [ ]:
pred_df

,TaskName,Unnamed: 0,l,g,b,t,o
0,LGBTphobiaDetectionMultiLabeled,0,X,G,X,X,X
1,LGBTphobiaDetectionMultiLabeled,1,X,G,X,X,X
2,LGBTphobiaDetectionMultiLabeled,2,X,G,X,X,X
3,LGBTphobiaDetectionMultiLabeled,3,X,G,X,X,X
4,LGBTphobiaDetectionMultiLabeled,4,X,G,X,X,X
...,...,...,...,...,...,...,...
472,LGBTphobiaDetectionMultiLabeled,472,X,G,X,X,X
473,LGBTphobiaDetectionMultiLabeled,473,X,X,X,T,X
474,LGBTphobiaDetectionMultiLabeled,474,X,X,X,T,X
475,LGBTphobiaDetectionMultiLabeled,475,X,X,X,T,X


In [ ]:
import csv
pred_df.to_csv('./track2_submission.txt', header=False, sep='\t', index=False, lineterminator='\n\n', quotechar='"', quoting=csv.QUOTE_ALL)